# Curva Mercado Local

In [1]:
import autograd.numpy as np
from autograd import grad
from functools import partial
from bisect import bisect_right
from finrisk import QC_Financial_3 as Qcf

**Primer Paso del Bootstrapping Local**

- Calculo E(ICP6M) con la cotización del swap a 6M
- Tengo las cotizaciones de swap y basis a 1Y:
  - Basis: $USDCLP \cdot V_{usd}=PV(discount_{clp},proyeccion_{clp})$
  
  - Swap: $PV_{fija}(discount_{clp})=PV(discount_{clp},proyeccion_{clp})$
  
  - Entonces: $PV_{fija}(discount_{clp})=V_{usd}$
  
  - Finalmente: Resuelvo la ecuación del basis usando la curva de descuento que acabo de obtener. Esta ecuación es de 1 incógnita porque ya tengo el E(ICP6M).

In [2]:
swap_6m = .000495
v_usd = 101.0
swap_1y = .000515
basis_1y = .00032

In [3]:
def lin_interpol(tenors, rates, tenor):
    if tenor >= tenors[len(tenors) - 1]:
        return rates[len(tenors) - 1]
    elif tenor <= tenors[0]:
        return rates[0]
    else:
        i = bisect_right(tenors, tenor) - 1
        m = (rates[i + 1] - rates[i]) / (tenors[i + 1] - tenors[i])
        return rates[i] + m * (tenor - tenors[i])
    
def df(rate, tenor):
    return (1 + rate)**(-tenor / 365.0)

def present_value(interp, disc, cashflow, tenor, tenors, rates):
    rate = interp(tenors, rates, tenor)
    return disc(rate, tenor) * cashflow

def kron(i, j):
    return int(i == j)

def dfs(rates, tenors):
    return np.array([df(z[0], z[1]) for z in zip(rates, tenors)])

def lin_interpols(tenors, rates, new_tenors):
    return np.array([lin_interpol(tenors, rates, t) for t in new_tenors])

def fixed_rate_leg(nocional, tasa, num_cupones):
    return np.array([.5 * 365.0 * i for i in range(1, num_cupones + 1)]), \
        np.array([nocional * (kron(i, num_cupones) + tasa / 2.0)
                  for i in range(1, num_cupones + 1)])

def present_value_2(interp, disc, cashflows_tenors, cashflows, curve_tenors, curve_rates):
    cashflow_rates = interp(curve_tenors, curve_rates, cashflows_tenors)
    return np.dot(dfs(cashflow_rates, cashflows_tenors), cashflows)

def pv_fixed_leg(interp, disc, nocional, tasa, num_cupones, curve_tenors, curve_rates):
    plazos, flujos = fixed_rate_leg(nocional, tasa, num_cupones)
    return present_value_2(interp, disc, plazos, flujos, curve_tenors, curve_rates)

def solve_2(nocional, tasa, num_cupones, obj, tenors, rates, pv):
    """
    Con nocional, tasa y num_cupones se construye una pata fija con
    periodicidad semestral. Por ejemplo, si nocional = 100, tasa = 2.0%
    y num_cupones = 4, se obtienen 4 cupones semestrales los primeros
    pagan un interés de 1 y el último para nocional (100) más el último
    interés igual a 1.
    
    Obj es el valor presente que quieres que tenga la pata fija,
    
    tenors y rates son los plazos y tasas de la curva inicial.
    
    pv es la función que calcula el valor presente de la pata fija.
    
    La función retorna la última tasa de la curva de modo que el valor
    presente de la pata fija sea igual a obj.
    """
    epsilon = .000001
    g = grad(pv, argnum=4)
    rates_ = np.array([r for r in rates] + [0.0,])
    print(rates_)
    which = len(rates_) - 1
    print(f'which: {which}')
    delta = 1
    while delta > epsilon:
        q = (pv(nocional, tasa, num_cupones, tenors, rates_) - obj)
        q /= g(nocional, tasa, num_cupones, tenors, rates_)[which]
        r1 = rates_[which] - q
        print(r1)
        delta = abs(r1 - rates_[which])
        print(f'delta: {delta}')
        if type(r1) is np.float64:
            rates_ = np.array([r for r in rates] + [r1,])
        else:
            rates_ = np.array([r for r in rates] + [r1._value,])
    return r1

In [4]:
tenors0 = np.array([1.0, 365.0])
rates0 = np.array([.0005,])

In [5]:
pv = partial(pv_fixed_leg, lin_interpols, dfs)

In [6]:
solve_2(100.0, swap_1y, 2, 100.5, tenors0, rates0, pv)

[0.0005 0.    ]
which: 1
-0.004483590015468371
delta: 0.004483590015468371
-0.00446357836567684
delta: 2.001164979153168e-05
-0.004463577963415107
delta: 4.022617326043809e-10


-0.004463577963415107

df * E(rswap - fwd) = 0

V = N(t) * E(F(T) / N(T))
V = OIS(t)*FX(t) * E(F(T) / (FX(T) * OIS(T))

In [7]:
tenors0 = np.array([1.0, 365.0, 730.0])
rates0 = np.array([.0005, -0.004463577963415107])

In [8]:
solve_2(100.0, swap_1y + .0003, 4, 101.0, tenors0, rates0, pv)

[ 0.0005     -0.00446358  0.        ]
which: 2
-0.0041808548075400916
delta: 0.0041808548075400916
-0.004154820185207878
delta: 2.603462233221384e-05
-0.004154819164283306
delta: 1.0209245714445925e-09


-0.004154819164283306

In [9]:
tenors0 = np.array([1.0, 365.0, 730.0])
rates0 = np.array([.0005, -0.004463577963415107, -0.004154819164283306])

**Pata ICP u OIS**

In [10]:
def piece_lin_interpol(tenors, rates, tenor):
    if tenor >= tenors[len(tenors) - 1]:
        return rates[len(tenors) - 1]
    elif tenor <= tenors[0]:
        return rates[0]
    else:
        i = bisect_right(tenors, tenor) - 1
        return rates[i]

In [11]:
tenors = np.array([0.0, 182.0, 365.0, 730.0])
rates = np.array([.0005, .0006, .0007,.0008])

In [12]:
piece_lin_interpol(tenors, rates, 730)

0.0008

In [13]:
grad(piece_lin_interpol, argnum=1)(tenors, rates, 30)

array([1., 0., 0., 0.])

In [14]:
def fwd_wf(tenors, rates, tenor1, tenor2):
    if tenor1 > tenor2:
        tenor1, tenor2 = tenor2, tenor1
    if tenor1 < tenors[0]:
        i1 = 0
    else:
        i1 = bisect_right(tenors, tenor1) - 1
    i2 = bisect_right(tenors, tenor2) - 1
    j = i1
    aux = [float(tenor1), float(tenor2)]
    while j <= i2:
        aux += [tenors[j]]
        j += 1
    aux = sorted(list(set(aux)))
    # print(aux)
    if aux[0] < tenor1:
        aux = aux[1:]
    # print(aux)
    wf = 1.0
    for j in range(len(aux) - 1):
        r = piece_lin_interpol(tenors, rates, aux[j])
        wf *= 1 + r * (aux[j + 1] - aux[j]) / 360.0
    return wf

In [15]:
fwd_wf(tenors, rates, 90, 200)

1.0001577816111111

In [16]:
(1+.0005*(182-100)/360)*(1+.0006*(365-182)/360)*(1+.0007*(730-365)/360)*(1+.0008*(750-730)/360)

1.0011734377863248

In [17]:
grad(fwd_wf, argnum=1)(tenors, rates, 365, 546)

array([0.        , 0.        , 0.50277778, 0.        ])

**Pata OIS**

In [18]:
def ois_leg(nocional, fecha_inicio, periodicidad, num_cupones, tenors, rates):
    fechas = []
    meses = periodicidad.get_months()
    for i in range(num_cupones):
        fechas.append((fecha_inicio.add_months(i * meses), fecha_inicio.add_months((i + 1) * meses)))
    plazos = []
    for f in fechas:
        plazos.append((float(fecha_inicio.day_diff(f[0])), float(fecha_inicio.day_diff(f[1]))))
    cashflows = []
    for p in plazos:
        wf = fwd_wf(tenors, rates, p[0], p[1])
        cashflows.append(nocional * (wf - 1))
    cashflows[len(cashflows) - 1] += nocional
    return np.array([p[1] for p in plazos]), np.array(cashflows)

In [19]:
p, c = ois_leg(100, Qcf.QCDate(17, 9, 2020), Qcf.Tenor('6M'), 4, tenors, rates)
p

array([181., 365., 546., 730.])

In [20]:
fwd_wf(tenors, rates, 0, 181)

1.0002513888888889

In [21]:
def pv_ois_leg(interp, disc, nocional, fecha_inicio, periodicidad,
               num_cupones, p_tenors, p_rates, d_tenors, d_rates):
    plazos, flujos = ois_leg(nocional, fecha_inicio, periodicidad, num_cupones, p_tenors, p_rates)
    return present_value_2(interp, disc, plazos, flujos, d_tenors, d_rates)

In [22]:
d_tenors = np.array([1.0, 365.0, 730.0])
d_rates = np.array([.0005, -0.004463577963415107, -0.004154819164283306])

p_tenors = np.array([1.0, 182.0, 365.0])
p_rates = np.array([.0005, -0.00055, -0.0006])

In [23]:
pv_ois_leg(lin_interpols, dfs, 100, Qcf.QCDate(17, 9, 2020), Qcf.Tenor('6M'), 4, p_tenors, p_rates,
          d_tenors, d_rates)

100.77210521335465